In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="5"

In [14]:
import itertools
import time
import numpy as np
import pandas as pd
import random
import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import  TensorDataset, DataLoader

import os
import copy
import math

from scipy.special import softmax
import scipy.stats as ss
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score,accuracy_score,confusion_matrix,recall_score,precision_score,precision_recall_curve,f1_score,auc

import matplotlib.pyplot as plt
import seaborn as sns; sns.set_theme(color_codes=True)

random_seed = 0
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)



skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
skf2 = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)

In [3]:

class EarlyStopping:
    def __init__(self, patience=7, verbose=False, delta=0):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.best_model = None

    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.best_model = model
            self.val_loss_min = val_loss
        elif score < self.best_score - self.delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.val_loss_min = val_loss
            self.best_model = model
            


In [4]:
# Number of Module = 16

class mtlAttention(nn.Module):
    def __init__(self, In_Nodes1,In_Nodes2, Modules):
        super(mtlAttention, self).__init__()
        self.Modules = Modules
        self.sigmoid = nn.Sigmoid()

        self.task1_FC1_x = nn.Linear(In_Nodes1, Modules,bias=False)
        self.task1_FC1_y = nn.Linear(In_Nodes1, Modules,bias=False)

        self.task2_FC1_x = nn.Linear(In_Nodes2, Modules,bias=False)
        self.task2_FC1_y = nn.Linear(In_Nodes2, Modules,bias=False)
            
        self.softmax  = nn.Softmax(dim=-1)
        
        self.task1_FC2 =nn.Sequential(nn.Linear(Modules*2, 16),nn.ReLU())
        self.task2_FC2 = nn.Sequential(nn.Linear(Modules*2, 16),nn.ReLU())
        
        self.task1_FC3 = nn.Sequential(nn.Linear(16, 1), nn.Sigmoid())
        self.task2_FC3 = nn.Sequential(nn.Linear(16, 1), nn.Sigmoid())

    def forward_one(self,xg,xm):
        xg_x = self.task1_FC1_x(xg)
        xm_x = self.task2_FC1_x(xm)
        xg_y = self.task1_FC1_y(xg)         
        xm_y =self.task2_FC1_y(xm)

        xg = torch.cat([xg_x.reshape(-1,1,self.Modules),xg_y.reshape(-1,1,self.Modules)], dim=1)
        xm = torch.cat([xm_x.reshape(-1,1,self.Modules),xm_y.reshape(-1,1,self.Modules)], dim=1)
        
        norm  = torch.norm(xg, dim=1, keepdim=True)
        xg = xg.div(norm)
        
        norm  = torch.norm(xm, dim=1, keepdim=True)
        xm = xm.div(norm)
        
        energy =  torch.bmm(xg.reshape(-1,2,self.Modules).permute(0,2,1) ,xm.reshape(-1,2,self.Modules))
        attention1 = self.softmax(energy.permute(0,2,1)).permute(0,2,1) 
        attention2 = self.softmax(energy).permute(0,2,1)
        
        xg_value = torch.bmm(xg,attention1) 
        xm_value = torch.bmm(xm,attention2)

        xg = xg_value.view(-1,self.Modules*2)
        xm =xm_value.view(-1,self.Modules*2)
        
        xg = self.task1_FC2(xg)
        xm = self.task2_FC2(xm) 
        xg = self.task1_FC3(xg)
        xm = self.task2_FC3(xm)
        
        return xg,xm

In [5]:
# Number of Module = 32

class mtlAttention(nn.Module):
    def __init__(self, In_Nodes1,In_Nodes2, Modules):
        super(mtlAttention, self).__init__()
        self.Modules = Modules
        self.sigmoid = nn.Sigmoid()

        self.task1_FC1_x = nn.Linear(In_Nodes1, Modules,bias=False)
        self.task1_FC1_y = nn.Linear(In_Nodes1, Modules,bias=False)

        self.task2_FC1_x = nn.Linear(In_Nodes2, Modules,bias=False)
        self.task2_FC1_y = nn.Linear(In_Nodes2, Modules,bias=False)
            
        self.softmax  = nn.Softmax(dim=-1)
        
        self.task1_FC2 =nn.Sequential(nn.Linear(Modules*2, 32),nn.ReLU())
        self.task2_FC2 = nn.Sequential(nn.Linear(Modules*2, 32),nn.ReLU())

        self.task1_FC3 =nn.Sequential(nn.Linear(32, 16),nn.ReLU())
        self.task2_FC3 = nn.Sequential(nn.Linear(32, 16),nn.ReLU())
        
        self.task1_FC4 = nn.Sequential(nn.Linear(16, 1), nn.Sigmoid())
        self.task2_FC4 = nn.Sequential(nn.Linear(16, 1), nn.Sigmoid())

    def forward_one(self,xg,xm):
        xg_x = self.task1_FC1_x(xg)
        xm_x = self.task2_FC1_x(xm)
        xg_y = self.task1_FC1_y(xg)         
        xm_y =self.task2_FC1_y(xm)

        xg = torch.cat([xg_x.reshape(-1,1,self.Modules),xg_y.reshape(-1,1,self.Modules)], dim=1)
        xm = torch.cat([xm_x.reshape(-1,1,self.Modules),xm_y.reshape(-1,1,self.Modules)], dim=1)
        
        norm  = torch.norm(xg, dim=1, keepdim=True)
        xg = xg.div(norm)
        
        norm  = torch.norm(xm, dim=1, keepdim=True)
        xm = xm.div(norm)
        
        energy =  torch.bmm(xg.reshape(-1,2,self.Modules).permute(0,2,1) ,xm.reshape(-1,2,self.Modules))
        attention1 = self.softmax(energy.permute(0,2,1)).permute(0,2,1) 
        attention2 = self.softmax(energy).permute(0,2,1)
        
        xg_value = torch.bmm(xg,attention1) 
        xm_value = torch.bmm(xm,attention2)

        xg = xg_value.view(-1,self.Modules*2)
        xm =xm_value.view(-1,self.Modules*2)
        
        xg = self.task1_FC2(xg)
        xm = self.task2_FC2(xm) 
        xg = self.task1_FC3(xg)
        xm = self.task2_FC3(xm)
        xg = self.task1_FC4(xg)
        xm = self.task2_FC4(xm)
        
        return xg,xm

In [6]:
# Number of Module = 64

class mtlAttention(nn.Module):
    def __init__(self, In_Nodes1,In_Nodes2, Modules):
        super(mtlAttention, self).__init__()
        self.Modules = Modules
        self.sigmoid = nn.Sigmoid()

        self.task1_FC1_x = nn.Linear(In_Nodes1, Modules,bias=False)
        self.task1_FC1_y = nn.Linear(In_Nodes1, Modules,bias=False)

        self.task2_FC1_x = nn.Linear(In_Nodes2, Modules,bias=False)
        self.task2_FC1_y = nn.Linear(In_Nodes2, Modules,bias=False)
            
        self.softmax  = nn.Softmax(dim=-1)
        
        self.task1_FC2 =nn.Sequential(nn.Linear(Modules*2, 64),nn.ReLU())
        self.task2_FC2 = nn.Sequential(nn.Linear(Modules*2, 64),nn.ReLU())

        self.task1_FC3 =nn.Sequential(nn.Linear(64, 32),nn.ReLU())
        self.task2_FC3 = nn.Sequential(nn.Linear(64, 32),nn.ReLU())
        
        self.task1_FC4 =nn.Sequential(nn.Linear(32, 16),nn.ReLU())
        self.task2_FC4 = nn.Sequential(nn.Linear(32, 16),nn.ReLU())
        
        self.task1_FC5 = nn.Sequential(nn.Linear(16, 1), nn.Sigmoid())
        self.task2_FC5 = nn.Sequential(nn.Linear(16, 1), nn.Sigmoid())

    def forward_one(self,xg,xm):
        xg_x = self.task1_FC1_x(xg)
        xm_x = self.task2_FC1_x(xm)
        xg_y = self.task1_FC1_y(xg)         
        xm_y =self.task2_FC1_y(xm)

        xg = torch.cat([xg_x.reshape(-1,1,self.Modules),xg_y.reshape(-1,1,self.Modules)], dim=1)
        xm = torch.cat([xm_x.reshape(-1,1,self.Modules),xm_y.reshape(-1,1,self.Modules)], dim=1)
        
        norm  = torch.norm(xg, dim=1, keepdim=True)
        xg = xg.div(norm)
        
        norm  = torch.norm(xm, dim=1, keepdim=True)
        xm = xm.div(norm)
        
        energy =  torch.bmm(xg.reshape(-1,2,self.Modules).permute(0,2,1) ,xm.reshape(-1,2,self.Modules))
        attention1 = self.softmax(energy.permute(0,2,1)).permute(0,2,1) 
        attention2 = self.softmax(energy).permute(0,2,1)
        
        xg_value = torch.bmm(xg,attention1) 
        xm_value = torch.bmm(xm,attention2)

        xg = xg_value.view(-1,self.Modules*2)
        xm =xm_value.view(-1,self.Modules*2)
        
        xg = self.task1_FC2(xg)
        xm = self.task2_FC2(xm) 
        xg = self.task1_FC3(xg)
        xm = self.task2_FC3(xm)
        xg = self.task1_FC4(xg)
        xm = self.task2_FC4(xm)
        xg = self.task1_FC5(xg)
        xm = self.task2_FC5(xm)
        
        return xg,xm

In [7]:
# Number of Module =128

class mtlAttention(nn.Module):
    def __init__(self, In_Nodes1,In_Nodes2, Modules):
        super(mtlAttention, self).__init__()
        self.Modules = Modules
        self.sigmoid = nn.Sigmoid()

        self.task1_FC1_x = nn.Linear(In_Nodes1, Modules,bias=False)
        self.task1_FC1_y = nn.Linear(In_Nodes1, Modules,bias=False)

        self.task2_FC1_x = nn.Linear(In_Nodes2, Modules,bias=False)
        self.task2_FC1_y = nn.Linear(In_Nodes2, Modules,bias=False)
            
        self.softmax  = nn.Softmax(dim=-1)
        
        self.task1_FC2 =nn.Sequential(nn.Linear(Modules*2, 128),nn.ReLU())
        self.task2_FC2 = nn.Sequential(nn.Linear(Modules*2, 128),nn.ReLU())

        self.task1_FC3 =nn.Sequential(nn.Linear(128, 64),nn.ReLU())
        self.task2_FC3 = nn.Sequential(nn.Linear(128, 64),nn.ReLU())
        
        self.task1_FC4 =nn.Sequential(nn.Linear(64, 32),nn.ReLU())
        self.task2_FC4 = nn.Sequential(nn.Linear(64, 32),nn.ReLU())
        
        self.task1_FC5 =nn.Sequential(nn.Linear(32, 16),nn.ReLU())
        self.task2_FC5 = nn.Sequential(nn.Linear(32, 16),nn.ReLU())
        
        self.task1_FC6 = nn.Sequential(nn.Linear(16, 1), nn.Sigmoid())
        self.task2_FC6 = nn.Sequential(nn.Linear(16, 1), nn.Sigmoid())

    def forward_one(self,xg,xm):
        xg_x = self.task1_FC1_x(xg)
        xm_x = self.task2_FC1_x(xm)
        xg_y = self.task1_FC1_y(xg)         
        xm_y =self.task2_FC1_y(xm)

        xg = torch.cat([xg_x.reshape(-1,1,self.Modules),xg_y.reshape(-1,1,self.Modules)], dim=1)
        xm = torch.cat([xm_x.reshape(-1,1,self.Modules),xm_y.reshape(-1,1,self.Modules)], dim=1)
        
        norm  = torch.norm(xg, dim=1, keepdim=True)
        xg = xg.div(norm)
        
        norm  = torch.norm(xm, dim=1, keepdim=True)
        xm = xm.div(norm)
        
        energy =  torch.bmm(xg.reshape(-1,2,self.Modules).permute(0,2,1) ,xm.reshape(-1,2,self.Modules))
        attention1 = self.softmax(energy.permute(0,2,1)).permute(0,2,1) 
        attention2 = self.softmax(energy).permute(0,2,1)
        
        xg_value = torch.bmm(xg,attention1) 
        xm_value = torch.bmm(xm,attention2)

        xg = xg_value.view(-1,self.Modules*2)
        xm =xm_value.view(-1,self.Modules*2)
        
        xg = self.task1_FC2(xg)
        xm = self.task2_FC2(xm) 
        xg = self.task1_FC3(xg)
        xm = self.task2_FC3(xm)
        xg = self.task1_FC4(xg)
        xm = self.task2_FC4(xm)
        xg = self.task1_FC5(xg)
        xm = self.task2_FC5(xm)
        xg = self.task1_FC6(xg)
        xm = self.task2_FC6(xm)
        
        return xg,xm

In [8]:
x1 = "./mRNA.csv"
x2 = "./meth.csv"
adj_file = "./bipartite.csv"
label = "./label.csv"

x1 = pd.read_csv(x1,index_col=0,delimiter=',')
x2 = pd.read_csv(x2, index_col=0, delimiter=',')
adj = pd.read_csv(adj_file, index_col=0)
xy, x_ind, y_ind = np.intersect1d(x1.columns, x2.columns, return_indices=True)
#_, x_ind1, y_ind1 = np.intersect1d(x2.index, adj.columns, return_indices=True)
#xy1, x_ind2, y_ind2 = np.intersect1d(x1.index, adj.index, return_indices=True)

x1 = x1.iloc[:, x_ind]
x2 = x2.iloc[:, y_ind]

x1 = x1.fillna(0)
x2 = x2.fillna(0)

data = pd.read_csv(label, delimiter=',', index_col=0)
xy, x_ind, y_ind = np.intersect1d(x1.columns, data.index, return_indices=True)
x1 = x1.iloc[:, x_ind]
x2 = x2.iloc[:, x_ind]
y = data.iloc[y_ind, :].astype(str)
# y[y=='Positive']=1
# y[y=='Negative']=0
labels = np.array(y).astype(np.float32)
# ER = 0 / TN = 1
labels = labels[:,1]

In [9]:
auc1 = []
auc2 = []
for fold_ in range(1,6):
    train_index = pd.read_csv('./train_index' + str(fold_) + '.csv', delimiter=',',header=None)
    valid_index = pd.read_csv('./valid_index' + str(fold_) + '.csv', delimiter=',',header=None)
    test_index = pd.read_csv('./test_index' + str(fold_) + '.csv', delimiter=',',header=None)
    
    train_index = np.array(train_index).reshape(-1).astype(int)
    valid_index = np.array(valid_index).reshape(-1).astype(int)
    test_index = np.array(test_index).reshape(-1).astype(int)
    
    Xg_train = x1.transpose().iloc[train_index,:].values
    Xg_valid = x1.transpose().iloc[valid_index,:].values
    Xg_test = x1.transpose().iloc[test_index,:].values

    Xm_train = x2.transpose().iloc[train_index,:].values
    Xm_valid = x2.transpose().iloc[valid_index,:].values
    Xm_test = x2.transpose().iloc[test_index,:].values

    y_train = labels[train_index]
    y_valid = labels[valid_index]
    y_test = labels[test_index]
    
    start = time.time()
    earlyStoppingPatience = 100
    learningRate = 0.000005
    weightDecay = 0.001
    num_epochs = 500000 

    y_train = y_train.astype(int)
    y_valid = y_valid.astype(int)
    y_test = y_test.astype(int)

    Xg = torch.tensor(Xg_train, dtype=torch.float32).cuda()
    Xm = torch.tensor(Xm_train, dtype=torch.float32).cuda()
    
    Xg_valid = torch.tensor(Xg_valid, dtype=torch.float32).cuda()
    Xm_valid = torch.tensor(Xm_valid, dtype=torch.float32).cuda()
    

    Xg_test = torch.tensor(Xg_test, dtype=torch.float32).cuda()
    Xm_test = torch.tensor(Xm_test, dtype=torch.float32).cuda()

    y = torch.tensor(y_train, dtype=torch.float32).cuda()
    y_v = torch.tensor(y_valid, dtype=torch.float32).cuda()

    ds = TensorDataset(Xg, Xm,y)
    dv = TensorDataset(Xg_valid,Xm_valid,y_v)
    
    loader  = DataLoader(ds, batch_size=y_train.shape[0],shuffle=True)
    loader_v = DataLoader(dv, batch_size=y_valid.shape[0],shuffle=True)

   

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    In_Nodes1 = Xg_train.shape[1] 
    In_Nodes2 = Xm_train.shape[1]

    # mtlAttention(In_Nodes1,In_Nodes2, # of module)
    net = mtlAttention(In_Nodes1,In_Nodes2,32)
    net = net.to(device)
    early_stopping = EarlyStopping(patience=earlyStoppingPatience, verbose=False)
    optimizer = optim.Adam(net.parameters(), lr=learningRate, weight_decay=weightDecay)
    loss_fn = nn.BCELoss()

    
    train_losses =[]
    valid_losses = [] 
    
    for epoch in (range(num_epochs)):
        running_loss1 = 0.0
        running_loss2 = 0.0
        for i, data in enumerate(loader, 0):
            xg,xm, y = data
            output1,output2 = net.forward_one(xg,xm)
            output1  = output1.squeeze()
            output2  = output2.squeeze()
            net.train()
            optimizer.zero_grad()
            loss = loss_fn(output1, y) + loss_fn(output2, y) 
            loss.backward(retain_graph=True)
            optimizer.step()
            running_loss1 += loss_fn(output1,y.view(-1)).item()
            running_loss2 += loss_fn(output2,y.view(-1)).item()
            train_losses.append(running_loss1+running_loss2)
        with torch.no_grad():
            net.eval()
            for i, data in enumerate(loader_v,0):
                xg,xm, y = data
                output1,output2 = net.forward_one(xg,xm)
                output1  = output1.squeeze()
                output2  = output2.squeeze()
                loss = loss_fn(output1, y) + loss_fn(output2, y)
                running_loss1 += loss_fn(output1,y.view(-1)).item()
                running_loss2 += loss_fn(output2,y.view(-1)).item()
                valid_losses.append(running_loss1+running_loss2)
        train_loss = np.average(train_losses)
        valid_loss = np.average(valid_losses)
        train_losses = []
        valid_losses = []
        
        
        early_stopping(valid_loss, net)
        if early_stopping.early_stop:
            print("Early stopping")
            print("--------------------------------------------------------------------------------------------------")
            break

        if (epoch+1) % 200 == 0 or epoch == 0:
            if (epoch+1) % 200 == 0 or epoch == 0:
                print ('Epoch [{}/{}], Loss: {:.4f}, BCE_task1; {:.4f}, BCE_task2; {:.4f}'.format(epoch+1,num_epochs, running_loss1+running_loss2,running_loss1,running_loss2))

    ### Test
    best_net = early_stopping.best_model
    test1,test2 = best_net.forward_one(Xg_test.clone().detach(),Xm_test.clone().detach())
    test1 = test1.cpu().detach().numpy()
    test2 = test2.cpu().detach().numpy()


    print ("ACC_task1 %.3f, ACC_task2 %.3f" %(accuracy_score(list(y_test),np.where(test1 > 0.5, 1, 0) ),accuracy_score(list(y_test),np.where(test2 > 0.5, 1, 0))))
    print ("F1_task1 %.3f, F1_task2 %.3f" %(f1_score(list(y_test),np.where(test1 > 0.5, 1, 0)),f1_score(list(y_test),np.where(test2 > 0.5, 1, 0))))
    print ("AUC_task1 %.3f, AUC_task2 %.3f" %(roc_auc_score(y_test.reshape(-1),test1),roc_auc_score(y_test.reshape(-1),test2)))
    print("time :", time.time() - start)
    auc1.append(roc_auc_score(y_test.reshape(-1),test1))
    auc2.append(roc_auc_score(y_test.reshape(-1),test2))
print("Task1 AUC: %.3f, Task2 AUC: %.3f" %(sum(auc1)/len(auc1), sum(auc2)/len(auc2)))

<ipython-input-9-002daccb3cdf>:53: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Xg_test = torch.tensor(Xg_test, dtype=torch.float32).cuda()
<ipython-input-9-002daccb3cdf>:54: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Xm_test = torch.tensor(Xm_test, dtype=torch.float32).cuda()


Epoch [1/500000], Loss: 2.6649, BCE_task1; 1.3981, BCE_task2; 1.2668
Epoch [200/500000], Loss: 2.6157, BCE_task1; 1.3703, BCE_task2; 1.2453
Epoch [400/500000], Loss: 2.5923, BCE_task1; 1.3587, BCE_task2; 1.2335
Epoch [600/500000], Loss: 2.5661, BCE_task1; 1.3469, BCE_task2; 1.2193
Epoch [800/500000], Loss: 2.5263, BCE_task1; 1.3322, BCE_task2; 1.1941
Epoch [1000/500000], Loss: 2.4812, BCE_task1; 1.3158, BCE_task2; 1.1654
Early stopping
--------------------------------------------------------------------------------------------------
ACC_task1 0.862, ACC_task2 0.862
F1_task1 0.926, F1_task2 0.926
AUC_task1 0.600, AUC_task2 0.590
time : 30.717185974121094


<ipython-input-9-002daccb3cdf>:53: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Xg_test = torch.tensor(Xg_test, dtype=torch.float32).cuda()
<ipython-input-9-002daccb3cdf>:54: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Xm_test = torch.tensor(Xm_test, dtype=torch.float32).cuda()


Epoch [1/500000], Loss: 2.8133, BCE_task1; 1.2541, BCE_task2; 1.5592
Epoch [200/500000], Loss: 2.7496, BCE_task1; 1.2142, BCE_task2; 1.5354
Epoch [400/500000], Loss: 2.7299, BCE_task1; 1.2010, BCE_task2; 1.5289
Epoch [600/500000], Loss: 2.7130, BCE_task1; 1.1882, BCE_task2; 1.5248
Early stopping
--------------------------------------------------------------------------------------------------
ACC_task1 0.862, ACC_task2 0.138
F1_task1 0.926, F1_task2 0.000
AUC_task1 0.330, AUC_task2 0.320
time : 17.0046284198761


<ipython-input-9-002daccb3cdf>:53: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Xg_test = torch.tensor(Xg_test, dtype=torch.float32).cuda()
<ipython-input-9-002daccb3cdf>:54: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Xm_test = torch.tensor(Xm_test, dtype=torch.float32).cuda()


Epoch [1/500000], Loss: 2.7146, BCE_task1; 1.1897, BCE_task2; 1.5249
Epoch [200/500000], Loss: 2.6734, BCE_task1; 1.1695, BCE_task2; 1.5038
Epoch [400/500000], Loss: 2.6588, BCE_task1; 1.1600, BCE_task2; 1.4987
Early stopping
--------------------------------------------------------------------------------------------------
ACC_task1 0.862, ACC_task2 0.138
F1_task1 0.926, F1_task2 0.000
AUC_task1 0.560, AUC_task2 0.540
time : 13.962572574615479
Epoch [1/500000], Loss: 2.9770, BCE_task1; 1.5129, BCE_task2; 1.4642


<ipython-input-9-002daccb3cdf>:53: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Xg_test = torch.tensor(Xg_test, dtype=torch.float32).cuda()
<ipython-input-9-002daccb3cdf>:54: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Xm_test = torch.tensor(Xm_test, dtype=torch.float32).cuda()


Epoch [200/500000], Loss: 2.9109, BCE_task1; 1.4802, BCE_task2; 1.4306
Epoch [400/500000], Loss: 2.8848, BCE_task1; 1.4693, BCE_task2; 1.4155
Epoch [600/500000], Loss: 2.8508, BCE_task1; 1.4555, BCE_task2; 1.3953
Epoch [800/500000], Loss: 2.8086, BCE_task1; 1.4390, BCE_task2; 1.3696
Epoch [1000/500000], Loss: 2.7647, BCE_task1; 1.4257, BCE_task2; 1.3390
Early stopping
--------------------------------------------------------------------------------------------------
ACC_task1 0.138, ACC_task2 0.862
F1_task1 0.000, F1_task2 0.920
AUC_task1 0.630, AUC_task2 0.620
time : 26.41229820251465
Epoch [1/500000], Loss: 2.8029, BCE_task1; 1.3842, BCE_task2; 1.4187

<ipython-input-9-002daccb3cdf>:53: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Xg_test = torch.tensor(Xg_test, dtype=torch.float32).cuda()
<ipython-input-9-002daccb3cdf>:54: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Xm_test = torch.tensor(Xm_test, dtype=torch.float32).cuda()



Epoch [200/500000], Loss: 2.7365, BCE_task1; 1.3492, BCE_task2; 1.3874
Epoch [400/500000], Loss: 2.7086, BCE_task1; 1.3364, BCE_task2; 1.3722
Early stopping
--------------------------------------------------------------------------------------------------
ACC_task1 0.862, ACC_task2 0.759
F1_task1 0.926, F1_task2 0.863
AUC_task1 0.330, AUC_task2 0.520
time : 15.877793312072754
Task1 AUC: 0.490, Task2 AUC: 0.518


With generated data

In [10]:
auc1 = []
auc2 = []
for fold_ in range(1,6):
    x_gen1 = pd.read_csv('./omics1_' + str(fold_) + '_' + '.csv', delimiter=',', index_col=0)
    x_gen2 = pd.read_csv('./omics2_' + str(fold_) + '_' + '.csv', delimiter=',', index_col=0)
    
    train_index = pd.read_csv('./train_index' + str(fold_) + '.csv', delimiter=',',header=None)
    valid_index = pd.read_csv('./valid_index' + str(fold_) + '.csv', delimiter=',',header=None)
    test_index = pd.read_csv('./test_index' + str(fold_) + '.csv', delimiter=',',header=None)
    
    train_index = np.array(train_index).reshape(-1).astype(int)
    valid_index = np.array(valid_index).reshape(-1).astype(int)
    test_index = np.array(test_index).reshape(-1).astype(int)
    
    Xg_train = pd.concat((x1.transpose().iloc[train_index,:],x_gen1)).values
    Xg_valid = x1.transpose().iloc[valid_index,:].values
    Xg_test = x1.transpose().iloc[test_index,:].values

    Xm_train = pd.concat((x2.transpose().iloc[train_index,:],x_gen2)).values
    Xm_valid = x2.transpose().iloc[valid_index,:].values
    Xm_test = x2.transpose().iloc[test_index,:].values

    y_train = labels[train_index]
    y_train = np.concatenate((y_train,y_train))
    y_valid = labels[valid_index]
    y_test = labels[test_index]
    
    start = time.time()
    earlyStoppingPatience = 100
    learningRate = 0.000005
    weightDecay = 0.001
    num_epochs = 500000 

    
    y_train = y_train.astype(int)
    y_valid = y_valid.astype(int)
    y_test = y_test.astype(int)

    Xg = torch.tensor(Xg_train, dtype=torch.float32).cuda()
    Xm = torch.tensor(Xm_train, dtype=torch.float32).cuda()
    
    Xg_valid = torch.tensor(Xg_valid, dtype=torch.float32).cuda()
    Xm_valid = torch.tensor(Xm_valid, dtype=torch.float32).cuda()
    

    Xg_test = torch.tensor(Xg_test, dtype=torch.float32).cuda()
    Xm_test = torch.tensor(Xm_test, dtype=torch.float32).cuda()

    y = torch.tensor(y_train, dtype=torch.float32).cuda()
    y_v = torch.tensor(y_valid, dtype=torch.float32).cuda()

    ds = TensorDataset(Xg, Xm,y)
    dv = TensorDataset(Xg_valid,Xm_valid,y_v)
    
    loader  = DataLoader(ds, batch_size=y_train.shape[0],shuffle=True)
    loader_v = DataLoader(dv, batch_size=y_valid.shape[0],shuffle=True)

    

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    In_Nodes1 = Xg_train.shape[1] 
    In_Nodes2 = Xm_train.shape[1]

    # mtlAttention(In_Nodes1,In_Nodes2, # of module)
    net = mtlAttention(In_Nodes1,In_Nodes2,32)
    net = net.to(device)
    early_stopping = EarlyStopping(patience=earlyStoppingPatience, verbose=False)
    optimizer = optim.Adam(net.parameters(), lr=learningRate, weight_decay=weightDecay)
    loss_fn = nn.BCELoss()

    
    train_losses =[]
    valid_losses = [] 
    
    for epoch in (range(num_epochs)):
        running_loss1 = 0.0
        running_loss2 = 0.0
        for i, data in enumerate(loader, 0):
            xg,xm, y = data
            output1,output2 = net.forward_one(xg,xm)
            output1  = output1.squeeze()
            output2  = output2.squeeze()
            net.train()
            optimizer.zero_grad()
            loss = loss_fn(output1, y) + loss_fn(output2, y) 
            loss.backward(retain_graph=True)
            optimizer.step()
            running_loss1 += loss_fn(output1,y.view(-1)).item()
            running_loss2 += loss_fn(output2,y.view(-1)).item()
            train_losses.append(running_loss1+running_loss2)
        with torch.no_grad():
            net.eval()
            for i, data in enumerate(loader_v,0):
                xg,xm, y = data
                output1,output2 = net.forward_one(xg,xm)
                output1  = output1.squeeze()
                output2  = output2.squeeze()
                loss = loss_fn(output1, y) + loss_fn(output2, y)
                running_loss1 += loss_fn(output1,y.view(-1)).item()
                running_loss2 += loss_fn(output2,y.view(-1)).item()
                valid_losses.append(running_loss1+running_loss2)
        train_loss = np.average(train_losses)
        valid_loss = np.average(valid_losses)
        train_losses = []
        valid_losses = []
        
        
        early_stopping(valid_loss, net)
        if early_stopping.early_stop:
            print("Early stopping")
            print("--------------------------------------------------------------------------------------------------")
            break

        if (epoch+1) % 200 == 0 or epoch == 0:
            if (epoch+1) % 200 == 0 or epoch == 0:
                print ('Epoch [{}/{}], Loss: {:.4f}, BCE_task1; {:.4f}, BCE_task2; {:.4f}'.format(epoch+1,num_epochs, running_loss1+running_loss2,running_loss1,running_loss2))

    ### Test
    best_net = early_stopping.best_model
    test1,test2 = best_net.forward_one(Xg_test.clone().detach(),Xm_test.clone().detach())
    test1 = test1.cpu().detach().numpy()
    test2 = test2.cpu().detach().numpy()


    print ("ACC_task1 %.3f, ACC_task2 %.3f" %(accuracy_score(list(y_test),np.where(test1 > 0.5, 1, 0) ),accuracy_score(list(y_test),np.where(test2 > 0.5, 1, 0))))
    print ("F1_task1 %.3f, F1_task2 %.3f" %(f1_score(list(y_test),np.where(test1 > 0.5, 1, 0)),f1_score(list(y_test),np.where(test2 > 0.5, 1, 0))))
    print ("AUC_task1 %.3f, AUC_task2 %.3f" %(roc_auc_score(y_test.reshape(-1),test1),roc_auc_score(y_test.reshape(-1),test2)))
    print("time :", time.time() - start)
    auc1.append(roc_auc_score(y_test.reshape(-1),test1))
    auc2.append(roc_auc_score(y_test.reshape(-1),test2))
print("Task1 AUC: %.3f, Task2 AUC: %.3f" %(sum(auc1)/len(auc1), sum(auc2)/len(auc2)))

<ipython-input-10-4f065e543b06>:58: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Xg_test = torch.tensor(Xg_test, dtype=torch.float32).cuda()
<ipython-input-10-4f065e543b06>:59: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Xm_test = torch.tensor(Xm_test, dtype=torch.float32).cuda()


Epoch [1/500000], Loss: 2.8898, BCE_task1; 1.4431, BCE_task2; 1.4467
Epoch [200/500000], Loss: 2.8508, BCE_task1; 1.4143, BCE_task2; 1.4365
Epoch [400/500000], Loss: 2.8264, BCE_task1; 1.3997, BCE_task2; 1.4267
Epoch [600/500000], Loss: 2.7983, BCE_task1; 1.3821, BCE_task2; 1.4162
Epoch [800/500000], Loss: 2.7642, BCE_task1; 1.3595, BCE_task2; 1.4046
Epoch [1000/500000], Loss: 2.7235, BCE_task1; 1.3320, BCE_task2; 1.3916
Epoch [1200/500000], Loss: 2.6779, BCE_task1; 1.3015, BCE_task2; 1.3763
Early stopping
--------------------------------------------------------------------------------------------------
ACC_task1 0.862, ACC_task2 0.724
F1_task1 0.926, F1_task2 0.833
AUC_task1 0.550, AUC_task2 0.480
time : 43.3552508354187


<ipython-input-10-4f065e543b06>:58: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Xg_test = torch.tensor(Xg_test, dtype=torch.float32).cuda()
<ipython-input-10-4f065e543b06>:59: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Xm_test = torch.tensor(Xm_test, dtype=torch.float32).cuda()


Epoch [1/500000], Loss: 3.1319, BCE_task1; 1.6157, BCE_task2; 1.5161
Epoch [200/500000], Loss: 3.0588, BCE_task1; 1.5891, BCE_task2; 1.4697
Epoch [400/500000], Loss: 3.0211, BCE_task1; 1.5762, BCE_task2; 1.4450
Epoch [600/500000], Loss: 2.9895, BCE_task1; 1.5622, BCE_task2; 1.4274
Epoch [800/500000], Loss: 2.9621, BCE_task1; 1.5497, BCE_task2; 1.4124
Early stopping
--------------------------------------------------------------------------------------------------
ACC_task1 0.138, ACC_task2 0.138
F1_task1 0.000, F1_task2 0.000
AUC_task1 0.150, AUC_task2 0.200
time : 29.23701310157776


<ipython-input-10-4f065e543b06>:58: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Xg_test = torch.tensor(Xg_test, dtype=torch.float32).cuda()
<ipython-input-10-4f065e543b06>:59: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Xm_test = torch.tensor(Xm_test, dtype=torch.float32).cuda()


Epoch [1/500000], Loss: 2.9783, BCE_task1; 1.6028, BCE_task2; 1.3754
Epoch [200/500000], Loss: 2.8891, BCE_task1; 1.5470, BCE_task2; 1.3421
Epoch [400/500000], Loss: 2.8545, BCE_task1; 1.5272, BCE_task2; 1.3274
Epoch [600/500000], Loss: 2.8162, BCE_task1; 1.5080, BCE_task2; 1.3082
Epoch [800/500000], Loss: 2.7800, BCE_task1; 1.4936, BCE_task2; 1.2864
Early stopping
--------------------------------------------------------------------------------------------------
ACC_task1 0.138, ACC_task2 0.862
F1_task1 0.000, F1_task2 0.926
AUC_task1 0.730, AUC_task2 0.790
time : 30.982171535491943


<ipython-input-10-4f065e543b06>:58: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Xg_test = torch.tensor(Xg_test, dtype=torch.float32).cuda()
<ipython-input-10-4f065e543b06>:59: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Xm_test = torch.tensor(Xm_test, dtype=torch.float32).cuda()


Epoch [1/500000], Loss: 2.9043, BCE_task1; 1.3825, BCE_task2; 1.5219
Epoch [200/500000], Loss: 2.8636, BCE_task1; 1.3735, BCE_task2; 1.4901
Epoch [400/500000], Loss: 2.8486, BCE_task1; 1.3676, BCE_task2; 1.4810
Epoch [600/500000], Loss: 2.8319, BCE_task1; 1.3607, BCE_task2; 1.4712
Epoch [800/500000], Loss: 2.8121, BCE_task1; 1.3527, BCE_task2; 1.4594
Epoch [1000/500000], Loss: 2.7895, BCE_task1; 1.3427, BCE_task2; 1.4467
Epoch [1200/500000], Loss: 2.7588, BCE_task1; 1.3284, BCE_task2; 1.4304
Epoch [1400/500000], Loss: 2.7195, BCE_task1; 1.3078, BCE_task2; 1.4118
Epoch [1600/500000], Loss: 2.6642, BCE_task1; 1.2738, BCE_task2; 1.3904
Early stopping
--------------------------------------------------------------------------------------------------
ACC_task1 0.862, ACC_task2 0.138
F1_task1 0.926, F1_task2 0.000
AUC_task1 0.690, AUC_task2 0.670
time : 54.83921790122986


<ipython-input-10-4f065e543b06>:58: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Xg_test = torch.tensor(Xg_test, dtype=torch.float32).cuda()
<ipython-input-10-4f065e543b06>:59: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Xm_test = torch.tensor(Xm_test, dtype=torch.float32).cuda()


Epoch [1/500000], Loss: 3.0922, BCE_task1; 1.5513, BCE_task2; 1.5409
Epoch [200/500000], Loss: 3.0041, BCE_task1; 1.5211, BCE_task2; 1.4829
Epoch [400/500000], Loss: 2.9719, BCE_task1; 1.5114, BCE_task2; 1.4605
Epoch [600/500000], Loss: 2.9356, BCE_task1; 1.5016, BCE_task2; 1.4340
Epoch [800/500000], Loss: 2.9091, BCE_task1; 1.4934, BCE_task2; 1.4157
Early stopping
--------------------------------------------------------------------------------------------------
ACC_task1 0.138, ACC_task2 0.138
F1_task1 0.000, F1_task2 0.000
AUC_task1 0.360, AUC_task2 0.450
time : 29.47883701324463
Task1 AUC: 0.496, Task2 AUC: 0.518


In [11]:
# Parameter ver.

In [12]:
module_list= [16,32,64,128]
lr_list = [0.000005,0.00005,0.0005,0.005]

In [22]:
auc1 = []
auc2 = []
for fold_ in range(1,6):
    train_index = pd.read_csv('./train_index' + str(fold_) + '.csv', delimiter=',',header=None)
    valid_index = pd.read_csv('./valid_index' + str(fold_) + '.csv', delimiter=',',header=None)
    test_index = pd.read_csv('./test_index' + str(fold_) + '.csv', delimiter=',',header=None)
    
    train_index = np.array(train_index).reshape(-1).astype(int)
    valid_index = np.array(valid_index).reshape(-1).astype(int)
    test_index = np.array(test_index).reshape(-1).astype(int)
    
    Xg_train = x1.transpose().iloc[train_index,:].values
    Xg_valid = x1.transpose().iloc[valid_index,:].values
    Xg_test = x1.transpose().iloc[test_index,:].values

    Xm_train = x2.transpose().iloc[train_index,:].values
    Xm_valid = x2.transpose().iloc[valid_index,:].values
    Xm_test = x2.transpose().iloc[test_index,:].values

    y_train = labels[train_index]
    y_valid = labels[valid_index]
    y_test = labels[test_index]
    
    
            
        
    
    earlyStoppingPatience = 100
    weightDecay = 0.001
    num_epochs = 500000 

    y_train = y_train.astype(int)
    y_valid = y_valid.astype(int)
    y_test = y_test.astype(int)

    Xg = torch.tensor(Xg_train, dtype=torch.float32).cuda()
    Xm = torch.tensor(Xm_train, dtype=torch.float32).cuda()
    
    Xg_valid = torch.tensor(Xg_valid, dtype=torch.float32).cuda()
    Xm_valid = torch.tensor(Xm_valid, dtype=torch.float32).cuda()
    

    Xg_test = torch.tensor(Xg_test, dtype=torch.float32).cuda()
    Xm_test = torch.tensor(Xm_test, dtype=torch.float32).cuda()

    y = torch.tensor(y_train, dtype=torch.float32).cuda()
    y_v = torch.tensor(y_valid, dtype=torch.float32).cuda()

    ds = TensorDataset(Xg, Xm,y)
    dv = TensorDataset(Xg_valid,Xm_valid,y_v)
    
    loader  = DataLoader(ds, batch_size=y_train.shape[0],shuffle=True)
    loader_v = DataLoader(dv, batch_size=y_valid.shape[0],shuffle=True)

    
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    In_Nodes1 = Xg_train.shape[1] 
    In_Nodes2 = Xm_train.shape[1]
    
    val_set = list(itertools.product(module_list,lr_list))
    best_valid = 99999
    for iter_ in tqdm.tqdm(range(0,len(val_set))):
        val_par = val_set[iter_]
        mod_num = val_par[0]
        learningRate = val_par[1]
        
    
        # mtlAttention(In_Nodes1,In_Nodes2, # of module)
        net = mtlAttention(In_Nodes1,In_Nodes2,mod_num)
        net = net.to(device)
        early_stopping = EarlyStopping(patience=earlyStoppingPatience, verbose=False)
        optimizer = optim.Adam(net.parameters(), lr=learningRate, weight_decay=weightDecay)
        loss_fn = nn.BCELoss()

    
        train_losses =[]
        valid_losses = [] 
    
        for epoch in (range(num_epochs)):
            running_loss1 = 0.0
            running_loss2 = 0.0
            for i, data in enumerate(loader, 0):
                xg,xm, y = data
                output1,output2 = net.forward_one(xg,xm)
                output1  = output1.squeeze()
                output2  = output2.squeeze()
                net.train()
                optimizer.zero_grad()
                loss = loss_fn(output1, y) + loss_fn(output2, y) 
                loss.backward(retain_graph=True)
                optimizer.step()
                running_loss1 += loss_fn(output1,y.view(-1)).item()
                running_loss2 += loss_fn(output2,y.view(-1)).item()
                train_losses.append(running_loss1+running_loss2)
            with torch.no_grad():
                net.eval()
                for i, data in enumerate(loader_v,0):
                    xg,xm, y = data
                    output1,output2 = net.forward_one(xg,xm)
                    output1  = output1.squeeze()
                    output2  = output2.squeeze()
                    loss = loss_fn(output1, y) + loss_fn(output2, y)
                    running_loss1 += loss_fn(output1,y.view(-1)).item()
                    running_loss2 += loss_fn(output2,y.view(-1)).item()
                    valid_losses.append(running_loss1+running_loss2)
            train_loss = np.average(train_losses)
            valid_loss = np.average(valid_losses)
            train_losses = []
            valid_losses = []
            
            early_stopping(valid_loss, net)
            if early_stopping.early_stop:
            #    print("Early stopping")
            #    print("--------------------------------------------------------------------------------------------------")
                break

            #if (epoch+1) % 200 == 0 or epoch == 0:
             #   if (epoch+1) % 200 == 0 or epoch == 0:
             #       print ('Epoch [{}/{}], Loss: {:.4f}, BCE_task1; {:.4f}, BCE_task2; {:.4f}'.format(epoch+1,num_epochs, running_loss1+running_loss2,running_loss1,running_loss2))
            
            
        best_score = early_stopping.best_score
        if best_score < best_valid:
            best_valid = valid_loss
            save_mod = mod_num
            save_lr = learningRate
            
            model = early_stopping.best_model
            
            
            best_net = early_stopping.best_model
            test1,test2 = best_net.forward_one(Xg_test.clone().detach(),Xm_test.clone().detach())
            test1 = test1.cpu().detach().numpy()
            test2 = test2.cpu().detach().numpy()
            
            auc1_best = roc_auc_score(y_test.reshape(-1),test1)
            auc2_best = roc_auc_score(y_test.reshape(-1),test2)
    
    print ("AUC_task1 %.3f, AUC_task2 %.3f" %(auc1_best,auc2_best))
    auc1.append(auc1_best)
    auc2.append(auc2_best)
print("Task1 AUC: %.3f, Task2 AUC: %.3f" %(sum(auc1)/len(auc1), sum(auc2)/len(auc2)))         

  0%|          | 0/16 [00:00<?, ?it/s]

AUC_task1 0.560, AUC_task2 0.540


  0%|          | 0/16 [00:00<?, ?it/s]

AUC_task1 0.350, AUC_task2 0.340


  0%|          | 0/16 [00:00<?, ?it/s]

AUC_task1 0.610, AUC_task2 0.640


  0%|          | 0/16 [00:00<?, ?it/s]

AUC_task1 0.680, AUC_task2 0.630


100%|██████████| 16/16 [02:47<00:00, 10.45s/it]

AUC_task1 0.540, AUC_task2 0.610
Task1 AUC: 0.548, Task2 AUC: 0.552


In [23]:
auc1 = []
auc2 = []
for fold_ in range(1,6):
    
    x_gen1 = pd.read_csv('./omics1_' + str(fold_) + '_' + '.csv', delimiter=',', index_col=0)
    x_gen2 = pd.read_csv('./omics2_' + str(fold_) + '_' + '.csv', delimiter=',', index_col=0)
    
    train_index = pd.read_csv('./train_index' + str(fold_) + '.csv', delimiter=',',header=None)
    valid_index = pd.read_csv('./valid_index' + str(fold_) + '.csv', delimiter=',',header=None)
    test_index = pd.read_csv('./test_index' + str(fold_) + '.csv', delimiter=',',header=None)
    
    train_index = np.array(train_index).reshape(-1).astype(int)
    valid_index = np.array(valid_index).reshape(-1).astype(int)
    test_index = np.array(test_index).reshape(-1).astype(int)
    
    Xg_train = pd.concat((x1.transpose().iloc[train_index,:],x_gen1)).values
    Xg_valid = x1.transpose().iloc[valid_index,:].values
    Xg_test = x1.transpose().iloc[test_index,:].values

    Xm_train = pd.concat((x2.transpose().iloc[train_index,:],x_gen2)).values
    Xm_valid = x2.transpose().iloc[valid_index,:].values
    Xm_test = x2.transpose().iloc[test_index,:].values

    y_train = labels[train_index]
    y_train = np.concatenate((y_train,y_train))
    y_valid = labels[valid_index]
    y_test = labels[test_index]
    
    
            
        
    
    earlyStoppingPatience = 100
    weightDecay = 0.001
    num_epochs = 500000 

    y_train = y_train.astype(int)
    y_valid = y_valid.astype(int)
    y_test = y_test.astype(int)

    Xg = torch.tensor(Xg_train, dtype=torch.float32).cuda()
    Xm = torch.tensor(Xm_train, dtype=torch.float32).cuda()
    
    Xg_valid = torch.tensor(Xg_valid, dtype=torch.float32).cuda()
    Xm_valid = torch.tensor(Xm_valid, dtype=torch.float32).cuda()
    

    Xg_test = torch.tensor(Xg_test, dtype=torch.float32).cuda()
    Xm_test = torch.tensor(Xm_test, dtype=torch.float32).cuda()

    y = torch.tensor(y_train, dtype=torch.float32).cuda()
    y_v = torch.tensor(y_valid, dtype=torch.float32).cuda()

    ds = TensorDataset(Xg, Xm,y)
    dv = TensorDataset(Xg_valid,Xm_valid,y_v)
    
    loader  = DataLoader(ds, batch_size=y_train.shape[0],shuffle=True)
    loader_v = DataLoader(dv, batch_size=y_valid.shape[0],shuffle=True)

    
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    In_Nodes1 = Xg_train.shape[1] 
    In_Nodes2 = Xm_train.shape[1]
    
    val_set = list(itertools.product(module_list,lr_list))
    best_valid = 99999
    for iter_ in tqdm.tqdm(range(0,len(val_set))):
        val_par = val_set[iter_]
        mod_num = val_par[0]
        learningRate = val_par[1]
        
    
        # mtlAttention(In_Nodes1,In_Nodes2, # of module)
        net = mtlAttention(In_Nodes1,In_Nodes2,mod_num)
        net = net.to(device)
        early_stopping = EarlyStopping(patience=earlyStoppingPatience, verbose=False)
        optimizer = optim.Adam(net.parameters(), lr=learningRate, weight_decay=weightDecay)
        loss_fn = nn.BCELoss()

    
        train_losses =[]
        valid_losses = [] 
    
        for epoch in (range(num_epochs)):
            running_loss1 = 0.0
            running_loss2 = 0.0
            for i, data in enumerate(loader, 0):
                xg,xm, y = data
                output1,output2 = net.forward_one(xg,xm)
                output1  = output1.squeeze()
                output2  = output2.squeeze()
                net.train()
                optimizer.zero_grad()
                loss = loss_fn(output1, y) + loss_fn(output2, y) 
                loss.backward(retain_graph=True)
                optimizer.step()
                running_loss1 += loss_fn(output1,y.view(-1)).item()
                running_loss2 += loss_fn(output2,y.view(-1)).item()
                train_losses.append(running_loss1+running_loss2)
            with torch.no_grad():
                net.eval()
                for i, data in enumerate(loader_v,0):
                    xg,xm, y = data
                    output1,output2 = net.forward_one(xg,xm)
                    output1  = output1.squeeze()
                    output2  = output2.squeeze()
                    loss = loss_fn(output1, y) + loss_fn(output2, y)
                    running_loss1 += loss_fn(output1,y.view(-1)).item()
                    running_loss2 += loss_fn(output2,y.view(-1)).item()
                    valid_losses.append(running_loss1+running_loss2)
            train_loss = np.average(train_losses)
            valid_loss = np.average(valid_losses)
            train_losses = []
            valid_losses = []
            
            early_stopping(valid_loss, net)
            if early_stopping.early_stop:
            #    print("Early stopping")
            #    print("--------------------------------------------------------------------------------------------------")
                break

            #if (epoch+1) % 200 == 0 or epoch == 0:
             #   if (epoch+1) % 200 == 0 or epoch == 0:
             #       print ('Epoch [{}/{}], Loss: {:.4f}, BCE_task1; {:.4f}, BCE_task2; {:.4f}'.format(epoch+1,num_epochs, running_loss1+running_loss2,running_loss1,running_loss2))
            
            
        best_score = early_stopping.best_score
        if best_score < best_valid:
            best_valid = valid_loss
            save_mod = mod_num
            save_lr = learningRate
            
            model = early_stopping.best_model
            
            
            best_net = early_stopping.best_model
            test1,test2 = best_net.forward_one(Xg_test.clone().detach(),Xm_test.clone().detach())
            test1 = test1.cpu().detach().numpy()
            test2 = test2.cpu().detach().numpy()
            
            auc1_best = roc_auc_score(y_test.reshape(-1),test1)
            auc2_best = roc_auc_score(y_test.reshape(-1),test2)
    
    print ("AUC_task1 %.3f, AUC_task2 %.3f" %(auc1_best,auc2_best))
    auc1.append(auc1_best)
    auc2.append(auc2_best)
print("Task1 AUC: %.3f, Task2 AUC: %.3f" %(sum(auc1)/len(auc1), sum(auc2)/len(auc2)))         

100%|██████████| 16/16 [05:18<00:00, 19.88s/it]


AUC_task1 0.570, AUC_task2 0.550


100%|██████████| 16/16 [04:16<00:00, 16.03s/it]


AUC_task1 0.360, AUC_task2 0.330


100%|██████████| 16/16 [04:03<00:00, 15.19s/it]


AUC_task1 0.700, AUC_task2 0.700


100%|██████████| 16/16 [04:53<00:00, 18.37s/it]


AUC_task1 0.670, AUC_task2 0.720


100%|██████████| 16/16 [04:03<00:00, 15.19s/it]

AUC_task1 0.630, AUC_task2 0.610
Task1 AUC: 0.586, Task2 AUC: 0.582
